In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d misrakahmed/vegetable-image-dataset

 96% 514M/534M [00:01<00:00, 282MB/s]
100% 534M/534M [00:01<00:00, 284MB/s]


In [ ]:
import os
import zipfile

local_zip = '/content/vegetable-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/data')
zip_ref.close()

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/data/Vegetable Images/train"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/content/data/Vegetable Images/validation"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [ ]:
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  elif epoch >= 7 and epoch < 10:
    return 1e-5
  else:
    return 1e-6

# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [ ]:
callbacks = [
    tf.keras.callbacks.LearningRateScheduler(decay),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,mode="min",restore_best_weights=True,),
    PrintLR()
]

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix

from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dense, Dropout

In [ ]:
class_map = train_generator.class_indices

In [ ]:
IMAGE_SIZE = [224, 224]

VGG = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in VGG.layers:
    layer.trainable = False

x = VGG.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

prediction = Dense(15, activation='softmax')(x)

model = Model(inputs=VGG.input, outputs=prediction)
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

80150528/80134624 [==============================] - 0s 0us/step


In [ ]:
history = model.fit(train_generator, epochs=15, steps_per_epoch=468, 
                    validation_data = validation_generator, callbacks=callbacks,
                    verbose = 1, validation_steps=93)

Epoch 1/15
468/468 [==============================] - ETA: 0s - loss: 0.6082 - accuracy: 0.8286
Learning rate for epoch 1 is 0.0010000000474974513
468/468 [==============================] - 210s 449ms/step - loss: 0.6082 - accuracy: 0.8286 - val_loss: 0.3064 - val_accuracy: 0.9170 - lr: 0.0010
Epoch 2/15
468/468 [==============================] - ETA: 0s - loss: 0.4453 - accuracy: 0.8732
Learning rate for epoch 2 is 0.0010000000474974513
468/468 [==============================] - 209s 446ms/step - loss: 0.4453 - accuracy: 0.8732 - val_loss: 0.2370 - val_accuracy: 0.9298 - lr: 0.0010
Epoch 3/15
468/468 [==============================] - ETA: 0s - loss: 0.3511 - accuracy: 0.9003
Learning rate for epoch 3 is 0.0010000000474974513
468/468 [==============================] - 208s 443ms/step - loss: 0.3511 - accuracy: 0.9003 - val_loss: 0.1834 - val_accuracy: 0.9553 - lr: 0.0010
Epoch 4/15
468/468 [==============================] - ETA: 0s - loss: 0.2969 - accuracy: 0.9161
Learning rate for e

In [ ]:
model.save('/content/model.h5')